# Recitation 2: Transfer Function

## Objectives
    1. Define a nonlinear dynamic model in python
    2. Compute the equilibrium and the linearized equations of the nonlinear dynamics
    3. Obtain the transfer function from the linearized dynamics
    

## Python Control System Toolbox
The [python-control package](https://python-control.readthedocs.io/en/0.9.4/) is a set of python classes and functions that implement common operations for the analysis and design of feedback control systems. 
Throughout the remainder of this course, we will utilize various commands from the control system toolbox.

Unfortunatley, the python-control package is not installed at GW jupyterhub. Currently, GW Instructional Technology Lab is checking if it can be installed easily. Meanwhile, we have to install it at every session of jupyterhub by executing

In [1]:
pip install control

Note: you may need to restart the kernel to use updated packages.


## Nonlinear State Equation

In the modeling, the equations of motion are reformulated in to the state equation, given by $\dot x = f (t, x, u)$ and $y=g(t,x,u)$. 

This can be formulated in python by [control.NonlinearIOSystem()](https://python-control.readthedocs.io/en/0.9.4/generated/control.NonlinearIOSystem.html#control.NonlinearIOSystem) with the usuage of

```control.NonlinearIOSystem(updfcn, outfcn=None, params=None, **kwargs)```

where 

* `updfcn(t, x, u, params)` is a python function for $f(t,x,u)$ with an additional parameter
* `outfcn(t, x, u, params)` is a python function for $g(t,x,u)$ with an additional parameter

### Pendulum

Recall that the state equation of the pendulum is

\begin{align*}
\dot x & = f(t, x, u) = 
\begin{bmatrix} 
x_2 \\
-\frac{g}{l}\sin x_1 + \frac{1}{ml^2}u
\end{bmatrix}, \\
y & = g(t, x, u) = x_1.
\end{align*}

Suppose $m=\frac{1}{2\times 9.8^2}\mathrm{kg}$, and $l=9.8\mathrm{m}$.
This can be modeled by the above python command as follows.


In [7]:
import control
import numpy as np

params = {"m": 1/2/9.8**2, "l": 9.8, "g": 9.8}
def pendulum_eom(t, x, u, params):
    m = params['m']
    l = params['l']
    g = params['g']

    x_dot = np.array([x[1], -g/l*np.sin(x[0]) + 1/m/l**2*u[0]])
    return x_dot

def pendulum_output(t, x, u, params):
    return x[0]

# define a nonlinear dynamic model (the second and the third lines are optional)
sys_pendulum = control.NonlinearIOSystem(updfcn = pendulum_eom, outfcn=pendulum_output, \
                                         states=('angle', 'angular_velocity'), output=('angle'), inputs=('torque'), \
                                         name='pendulum')

print(sys_pendulum)


<NonlinearIOSystem>: pendulum
Inputs (1): ['torque']
Outputs (1): ['angle']
States (2): ['angle', 'angular_velocity']

Update: <function pendulum_eom at 0x13a278fe0>
Output: <function pendulum_output at 0x13a279440>


### Notes
There are a couple of details that should be satisfied, as summarized below.
* Both functions takes four input arguments of `(t, x, u, params)`
* While the input $u$ is one-dimensional, it is referenced by `u[0]` with an index `0`
* The output of the function is an numpy array.

## Transfer Function

The transfer function of the pendulum is obtained by the three steps of
1. Find equilibirum using [control.find_eqpt()](https://python-control.readthedocs.io/en/0.9.4/generated/control.find_eqpt.html).
2. Find the linearized equations of motion, or the linear state-spade model using [control.linearize()](https://python-control.readthedocs.io/en/latest/generated/control.linearize.html)
3. Find the trasfer function using [control.ss2tf()](https://python-control.readthedocs.io/en/latest/generated/control.ss2tf.html)
### Pendulum

For the above pendulum model, 

In [3]:
# initial guess of the equilibrium state and control
x0 = [0.1, 0.1]
u0 = [0]

# Find equilibrium state and control near (x0, u0)
x_star, u_star = control.find_eqpt(sys_pendulum, x0=x0, u0=[0], params=params)
print(f'Equilibrium state = {x_star}')
print(f'Equilibrium control = {u_star}')


Equilibrium state = [0. 0.]
Equilibrium control = [0]


In [4]:
# Linearize about the above equilibrium
ss_pendulum = control.linearize(sys_pendulum, x_star, u_star, params=params)
print(ss_pendulum)



<LinearIOSystem>: sys[3]
Inputs (1): ['u[0]']
Outputs (1): ['y[0]']
States (2): ['x[0]', 'x[1]']

A = [[ 0.  1.]
     [-1.  0.]]

B = [[0.]
     [2.]]

C = [[1. 0.]]

D = [[0.]]



In [5]:
# Transfer function
tf_pendulum = control.ss2tf(ss_pendulum)
print(tf_pendulum)


   2
-------
s^2 + 1



## Task

The above linearization and trasnfer function are about the **hanging** equilibrium of the pendulum. Repeat the above procedure to obtain the transfer function linearized about the **inverted** equilibrium.

The code largely remains unchanged. Copy and paste the above code, and revise the initial guess of the equilibrium state close to $[\pi, 0]$. (While we already know that the inverted equilibirum is at $[\pi, 0]$, we wish to rediscover that using Python here.)